在本教程中，你将学习三种**处理缺失值**的方法。随后我们会在一个真实数据集上比较这些方法的效果。

# 引言

数据出现缺失值的原因很多。例如：
- 一套 2 卧室的房子不会有第三间卧室面积的数值；
- 问卷参与者可能选择不透露其收入。

大多数机器学习库（包括 scikit-learn）在你使用包含缺失值的数据训练模型时会报错。因此，你需要在下面的策略中做出选择。

# 三种方法


### 1）简单选项：删除含缺失值的列

最简单的方法是删除包含缺失值的列。

![tut2_approach1](https://storage.googleapis.com/kaggle-media/learn/images/Sax80za.png)

除非被删除的列中绝大多数条目都是缺失的，否则这种做法会让模型丢失大量（可能很有用！）的信息。一个极端的例子是：假设一个有 10,000 行的数据集中，有一列很重要但仅缺失了 1 个值，这种做法会直接把整列都删掉！

### 2）更好的选项：插补（Imputation）

**插补**用某个数值来填充缺失值。例如，我们可以按列使用均值进行填充。

![tut2_approach2](https://storage.googleapis.com/kaggle-media/learn/images/4BpnlPA.png)

插补出来的值在大多数情况下不可能完全准确，但通常会比直接删除整列得到更准确的模型。

### 3）插补的扩展

插补是标准做法，而且通常有效。但插补值可能系统性地偏高或偏低（因为真实值并未被采集）；或者，含缺失值的行在其他方面也有其独特性。在这种情况下，如果模型能知道哪些值最初是缺失的，预测可能会更好。

![tut3_approach3](https://storage.googleapis.com/kaggle-media/learn/images/UWOyg4a.png)

在这种方法中，我们像之前一样先进行缺失值插补；此外，对原始数据集中每个含缺失的列，新增一个布尔列，指示哪些位置是被插补过的。

在某些情形下，这能显著提升效果；但在另一些情形下，这可能也毫无作用。

# 示例

在接下来的示例中，我们将使用 [墨尔本房价数据集](https://www.kaggle.com/dansbecker/melbourne-housing-snapshot/home)。我们的模型将使用房间数、土地面积等信息来预测房价。

我们不会展开数据加载步骤。你可以假设我们已经得到了训练/验证数据：`X_train`、`X_valid`、`y_train` 和 `y_valid`。

In [ ]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Load the data
data = pd.read_csv('../input/melbourne-housing-snapshot/melb_data.csv')

# Select target
y = data.Price

# To keep things simple, we'll use only numerical predictors
melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

# Divide data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

### 定义用于评估各方法质量的函数

我们定义函数 `score_dataset()` 来比较不同的缺失值处理方法。该函数返回随机森林模型的[平均绝对误差](https://en.wikipedia.org/wiki/Mean_absolute_error)（MAE）。

In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

### 方法一评分（删除含缺失值的列）

由于我们同时处理训练集与验证集，需要确保在两个 DataFrame 中删除的是同一批列。

In [ ]:
# Get names of columns with missing values
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]

# Drop columns in training and validation data
reduced_X_train = X_train.drop(cols_with_missing, axis=1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

### 方法二评分（插补）

接下来，我们使用 [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html) 按列均值来替换缺失值。

虽然做法简单，但用均值填充通常效果不错（不同数据集会有差异）。统计学上也有更复杂的插补方法（比如**回归插补**），但一旦将结果输入到复杂的机器学习模型中，这些复杂策略通常并不会带来额外收益。

In [ ]:
from sklearn.impute import SimpleImputer

# Imputation
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

我们可以看到，**方法二** 的 MAE 低于 **方法一**，因此在该数据集上 **方法二** 表现更好。

### 方法三评分（插补的扩展）

接下来，我们不仅进行插补，还记录哪些位置是被插补过的。

In [ ]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

如上所示，**方法三** 的表现略差于 **方法二**。

### 为什么插补比删除整列表现更好？

训练数据包含 10864 行与 12 列，其中有 3 列存在缺失。对每一列而言，缺失比例都小于一半。因此，删除整列会移除大量有用信息，所以插补往往表现更好是合理的。

In [ ]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

# 结论
如同常见情况，相比直接删除含缺失的列（**方法一**），对缺失值进行插补（**方法二**与**方法三**）通常能得到更好的结果。

# 轮到你了

在 **[这个练习](https://www.kaggle.com/kernels/fork/3370280)** 中亲自比较这些缺失值处理方法吧！

---




*有问题或想交流？欢迎访问[课程讨论区](https://www.kaggle.com/learn/intermediate-machine-learning/discussion)与其他学习者交流。*